In [1]:
# Load duckdb, which lets us efficiently load large files
import duckdb

# Load pandas, which lets us manipulate dataframes
import pandas as pd

# Import jupysql Jupyter extension to create SQL cells
%load_ext sql

# Set configrations on jupysql to directly output data to Pandas and to simplify the output that is printed to the notebook.
%config SqlMagic.autopandas = True

%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

# Connect jupysql to DuckDB using a SQLAlchemy-style connection string. Either connect to an in memory DuckDB, or a file backed db.
%sql duckdb:///:memory:

In [2]:
import duckdb
import pandas as pd
import glob
import os

# Connect to DuckDB
con = duckdb.connect(':memory:')

# Get list of parquet files
parquet_files = glob.glob('/Users/me/data/foodb/*.parquet')
tables = [os.path.splitext(os.path.basename(f))[0] for f in parquet_files]

def get_table_stats(table):
    # Get columns first to check what fields exist
    columns = con.execute(f"SELECT * FROM read_parquet('~/data/foodb/{table}.parquet') LIMIT 0").df().columns
    
    # Build dynamic parts of the query based on common join fields
    stats_columns = [
        "COUNT(*) AS total_rows",
        f"COUNT(DISTINCT id) AS unique_ids" if 'id' in columns else "NULL AS unique_ids",
        f"COUNT(DISTINCT food_id) AS unique_foods" if 'food_id' in columns else "NULL AS unique_foods",
        f"COUNT(DISTINCT compound_id) AS unique_compounds" if 'compound_id' in columns else "NULL AS unique_compounds",
        f"COUNT(DISTINCT flavor_id) AS unique_flavors" if 'flavor_id' in columns else "NULL AS unique_flavors"
    ]
    
    query = f"""
    SELECT 
        '{table}' AS table_name,
        {','.join(stats_columns)}
    FROM read_parquet('~/data/foodb/{table}.parquet')
    """
    
    return con.execute(query).fetchdf()

# Get stats for all tables
results = pd.concat([get_table_stats(table) for table in tables])

# Sort by table name
results = results.sort_values('table_name')

# Set display options for better readability
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
print("\nFoodDB Table Statistics:")
print("=" * 80)
print(results.to_string(index=False))

# Generate summary insights
print("\nSummary Insights:")
print("=" * 80)
print(f"Total number of tables: {len(results)}")
if 'unique_foods' in results.columns:
    food_tables = results[results['unique_foods'].notnull()]
    if not food_tables.empty:
        print(f"Tables with food relationships: {len(food_tables)}")
if 'unique_compounds' in results.columns:
    compound_tables = results[results['unique_compounds'].notnull()]
    if not compound_tables.empty:
        print(f"Tables with compound relationships: {len(compound_tables)}")
if 'unique_flavors' in results.columns:
    flavor_tables = results[results['unique_flavors'].notnull()]
    if not flavor_tables.empty:
        print(f"Tables with flavor relationships: {len(flavor_tables)}")

# Save results to CSV
results.to_csv('foodb_table_stats.csv', index=False)


FoodDB Table Statistics:
                table_name  total_rows  unique_ids  unique_foods  unique_compounds  unique_flavors
           AccessionNumber        1424        1424           NaN            1326.0             NaN
                  Compound       70477       70477           NaN               NaN             NaN
   CompoundAlternateParent       50691       50691           NaN            5225.0             NaN
CompoundExternalDescriptor        4009        4009           NaN            2553.0             NaN
      CompoundOntologyTerm     1587712     1587712           NaN           78133.0             NaN
       CompoundSubstituent       95299       95299           NaN            5262.0             NaN
           CompoundSynonym      171240      171240           NaN               NaN             NaN
           CompoundsEnzyme      105089      105089           NaN            5997.0             NaN
           CompoundsFlavor       11775       11775           NaN            2871.0 